# UrbanSound8K

In [2]:
import soundfile as sf

def obtener_info_audio(archivo_wav):
    info = sf.info(archivo_wav)
    tasa_muestreo = info.samplerate
    subtipo = info.subtype

    # Extraemos la profundidad en bits a partir del subtipo si contiene 'PCM'
    if 'PCM' in subtipo:
        try:
            bits = int(subtipo.split('_')[1])
        except (IndexError, ValueError):
            bits = "No disponible"
    else:
        bits = "No disponible"
    
    return tasa_muestreo, bits

if __name__ == "__main__":
    archivo_wav = "C:/Users/cadur/Downloads/Urbansonic/0_Mels_Gen/DATA/UrbanSound8K/audio/fold1/7383-3-1-0.wav"  
    tasa, bits = obtener_info_audio(archivo_wav)
    print(f"Tasa de muestreo del archivo WAV: {tasa} Hz")
    print(f"Resolución del audio (profundidad en bits): {bits} bits")

Tasa de muestreo del archivo WAV: 44100 Hz
Resolución del audio (profundidad en bits): 16 bits


In [ ]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los audios y subcarpetas
ruta_carpeta = 'C:/Users/cadur/Downloads/Urbansonic/0_Mels_Gen/DATA/ESC-50-master/audio/'

# Listas para almacenar nombres de audios, etiquetas y rutas
nombres_audios = []
etiquetas = []
rutas_audios = []

# Recorrer la carpeta y sus subcarpetas
for etiqueta in os.listdir(ruta_carpeta):
    carpeta_etiqueta = os.path.join(ruta_carpeta, etiqueta)

    # Verificar si es una subcarpeta
    if os.path.isdir(carpeta_etiqueta):
        for audio in os.listdir(carpeta_etiqueta):
            if audio.endswith('.wav'):  # Ajustar las extensiones según tus archivos
                ruta_audio = os.path.join(carpeta_etiqueta, audio)
                nombres_audios.append(audio)
                etiquetas.append(etiqueta)
                rutas_audios.append(ruta_audio)

# Crear un Marco de Datos con los datos recopilados
data = {'name': nombres_audios, 'class': etiquetas, 'audio_path': rutas_audios}
df = pd.DataFrame(data)

# Guardar el DataFrame en un archivo CSV
ruta_csv = 'C:/Users/cadur/Downloads/Urbansonic/0_Mels_Gen/DATA/ESC-50-master/csv/ESC_AllMels.csv'
df.to_csv(ruta_csv, index=False)

print(f"Archivo CSV generado y guardado en {ruta_csv}")

# Mels 2 segundos

In [ ]:
import os
import matplotlib
matplotlib.use('Agg') # No pictures displayed
import pylab
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import joblib
import math
import pandas as pd
import soundfile as sf
import wave

In [ ]:
# melspec dir from previous script
wav_dir = 'E:/VNE/Audios/Audios_VNE_2s/VNE_2seg'
# where the melspecs will be generated. Pathway needs to exist, next chunk will create class folders IN the folder
mel_spec_dir = "E:/VNE/Mels_Completo/Mels_New"

In [ ]:
# Read CSV file
csv_path = "E:/VNE/CSV/VNE_AllMels.csv"
csv_data = pd.read_csv(csv_path)

In [ ]:
# Extracting unique classes from the CSV
cutoff_class_name = csv_data['class'].unique()

# build a mel spec directory for each class
for i in range(len(cutoff_class_name)):
    class_dir = os.path.join(mel_spec_dir, cutoff_class_name[i])
    if os.path.exists(class_dir):
        print(cutoff_class_name[i], "directory already exists. Skipping...")
    else:
        os.mkdir(class_dir)

print("Created melspec dirs:")
print(os.listdir(mel_spec_dir))
print(len(os.listdir(mel_spec_dir)))

In [ ]:
SR = 22050  # Sampling frequency
FEATURE = 'mel'  # feature representation

FMIN = 0  # 250 # 1000
en = 0

roi_len = 5  # segmentacion de 5 segundos

n_sam = int(22050 * roi_len)
print(wav_dir)

In [ ]:
# generate mel specs for each audio file listed in the CSV
for index, row in csv_data.iterrows():
    wav_nm = row['name']
    class_label = row['class']
    audio_path = row['audio_path']

    print(wav_nm)
    print('Class:', class_label)

    # Cargar el archivo de audio en formato WAVEX
    audio_data, sample_rate = sf.read(audio_path)
    print(sample_rate)

    # Modificar el formato del archivo a WAV en memoria
    sf.write(audio_path, audio_data, sample_rate, format='WAV')

    # Ruta del archivo de audio .wav
    audio_con = audio_path

     # Cargar el archivo de audio y convertir a punto flotante
    with wave.open(audio_con) as audio:
        sample_rate = audio.getframerate()
        audio_data = np.frombuffer(audio.readframes(-1), dtype='int16')


    fig = plt.figure(1, frameon=False)
    fig.set_size_inches(6, 6)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    # Calcular el espectrograma
    espectrograma = librosa.feature.melspectrogram(y=audio_data.astype(np.float32), sr=sample_rate)

    # y, sr = librosa.load(audio_path, sr=48000)
    n_fft = 2048
    hop_length = 32
    n_mels = 128
    fmin = 0
    fmax = 11025

    S = librosa.feature.melspectrogram(y=audio_data.astype(np.float16), sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmin = fmin, fmax = fmax)

    librosa.display.specshow(librosa.power_to_db(S ** 1, ref=np.max), fmin=0, y_axis='linear')  # , cmap = 'gray')

    class_subdir = os.path.join(mel_spec_dir, class_label)
    directory = os.path.join(class_subdir, wav_nm.split('.')[0] + '.png')  # 'test.png'
    fig.savefig(directory)
    fig.clear()
    ax.cla()
    plt.clf()
    plt.close('all')

    print('Mel spectrogram saved:', directory)
    print('-' * 70)